In [1]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
import torchmetrics

from TorchTemplates import SimpleBinaryClassificationNN as sbcnn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_csv(r'C:/Users/User/Desktop/credit card/creditcard.csv').drop(columns=['Time'])

train_val, test = train_test_split(df, test_size=.15, stratify=df['Class'])
train, val = train_test_split(train_val, test_size=.2, stratify=train_val['Class'])

In [3]:
train.shape, val.shape, test.shape 

((193668, 30), (48417, 30), (42722, 30))

In [4]:
for i in (train, val, test):
    print(i['Class'].value_counts(normalize=True))

0    0.998275
1    0.001725
Name: Class, dtype: float64
0    0.998265
1    0.001735
Name: Class, dtype: float64
0    0.998268
1    0.001732
Name: Class, dtype: float64


In [5]:
target = 'Class'

X_train = train[train.columns.drop(target)]
X_val = val[val.columns.drop(target)]
X_test = test[test.columns.drop(target)]

y_train = train[target]
y_val = val[target]
y_test = test[target]

In [6]:
train_loader, scaler = sbcnn.make_loader(X_train, y_train, shuffle=True, batch_size=2048, fit_scaler=True, scaler=StandardScaler)
val_loader = sbcnn.make_loader(X_val, y_val, X_val.shape[0], scaler=scaler)
test_loader = sbcnn.make_loader(X_test, y_test, X_test.shape[0], scaler=scaler)

In [7]:
sbcnn.layer_template()


{
'input': {'units':[None,None], 'batch_norm': None, 'dropout': None, 'activation': None},
'fcn'  : {'units':[None,None], 'batch_norm': None, 'dropout': None, 'activation': None},
'output':{'units':[None,None], 'batch_norm': None, 'dropout': None, 'activation': None}
}



In [8]:
model = sbcnn({
'input':  {'units':[29,32], 'batch_norm': True, 'dropout': 0.5, 'activation': 'ReLU'},
'fc1'  :  {'units':[32,8], 'batch_norm': True, 'dropout': 0.3, 'activation': 'ReLU'},
'output': {'units':[8,1], 'batch_norm': None, 'dropout': 0.0, 'activation': None}
},
l1_lambda=0.01,
l2_lambda=0.01,
scaler=scaler)

In [9]:
lossfun = nn.BCEWithLogitsLoss(pos_weight = torch.tensor(5))
optimizer = optim.Adam(model.parameters(), lr = 0.01)